In [96]:
#criar navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
navegador = webdriver.Chrome()
import os

In [97]:
#abrir a página index, entrar no site da busca jurídica
caminho = os.getcwd()
arquivo = caminho + r"\Modulo 24 - Aula 25\index.html"


In [98]:
#importar a base de daos
import pandas as pd

tabela = pd.read_excel(r"Modulo 24 - Aula 25\Processos.xlsx")
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [99]:
#escolher um estado

from selenium.webdriver import ActionChains
import time


for linha in tabela.index:

    #para cada processo (linha da tabela)
    navegador.get(arquivo)

    #abrir a lista de estados
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()
    
    cidade = tabela.loc[linha, 'Cidade']
    #selecionando a cidade
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    #mudar para nova aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    aba_nova = navegador.window_handles[indice]

    navegador.switch_to.window(aba_nova)
    
    #preencher o formulário com os dados de busca
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, 'Nome'])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, 'Advogado'])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, 'Processo'])
    
    #clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    #confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()

    #esperar o resultado da pesquisa e agir de acordo com o resultado
    while True:
        try: 
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, "Status"] = "Encontrado"
    else:
        tabela.loc[linha, 'Status'] = "Não Encontrado"
        alerta.accept()

In [100]:
navegador.quit()
display(tabela)

tabela.to_excel("Processos Atualizados.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Não Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
